## Jax IlluminaWGS 25Aug, 2020

## Notes
1. -mq  -q means quiet, no verbose at all
2. gsutil doesn't work on python3.8, which is installed in my plot env
3. global varibles and libraries
4. Remeber to update the WRKDIR for previously used notebooks
5. Remember do not add comments before %%bash
6. Remember to replace 'usuhsID' with 'Sample_Name' in sample_ids
7. make sure the json or tool dir are there for the next steps

In [1]:
#global notebook variables for both python and bash majic (by stdin arguments)
#WRKDIR = '/Users/mooreank/Desktop/Mark/UNHS'
WRKDIR = '/Users/pengl7/Downloads/WGS/Jax'
#PRJ_BUCKET = 'gs://nihnialngcbg-testing'
PRJ_BUCKET = 'gs://test-7cee72c0e768'
#PROJECT_ID = 'nih-nia-lng-cbg'
PROJECT_ID = 'singlecellseq'
#MYUSER = 'mooreank'
MYUSER = 'pengl7'
AUTOSOMES=[str(x) for x in list(range(1,23))]
SEXOMES=['X','Y']
CHROMOSOMES=AUTOSOMES + SEXOMES

# COHORT='UNHS'
# remember the folder sequence should also be the same in the local dir
COHORT = 'Jax'

# Notice that COHORT_BUCKET means the main data_folder path in the project bucket, instead of a bucket name
COHORT_BUCKET=f'{PRJ_BUCKET}/{COHORT}'

 -s option: This option allows the positional parameters to be set when invoking an interactive shell.

In [2]:
%%bash -s "$PRJ_BUCKET" "$COHORT"
PRJ_BUCKET=${1}
COHORT=${2}
echo ${PRJ_BUCKET}/${COHORT}

gs://test-7cee72c0e768/Jax


In [3]:
import pandas as pd
import numpy as np
import time
import json
import os
import seaborn as sns
import matplotlib.pyplot as plt

#### Check the vcf files inside the data folder

In [6]:
#confirm output file was successfull created
!gsutil -mq ls -lh {COHORT_BUCKET}/hg38/joint-discovery

  2.94 KiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.variant_calling_detail_metrics
  1.52 KiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.variant_calling_summary_metrics
  1.68 GiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz
  2.55 MiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz.tbi
     748 B  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/out.intervals
904.95 KiB  2020-08-25T15:04:43Z  gs://test-7cee72c0e768/Jax/hg38/joint-discovery/wdl_run_metadata.json
TOTAL: 6 objects, 1802286842 bytes (1.68 GiB)


## Run QC on samples

In [8]:
!mkdir /Users/pengl7/Downloads/WGS/{COHORT}/genotypes
!gsutil -m cp {COHORT_BUCKET}/hg38/joint-discovery/*gz* /Users/pengl7/Downloads/WGS/{COHORT}/genotypes/

Copying gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://test-7cee72c0e768/Jax/hg38/joint-discovery/Jax.vcf.gz.tbi...
/ [2/2 files][  1.7 GiB/  1.7 GiB] 100% Done   8.2 MiB/s ETA 00:00:00           
Operation completed over 2 objects/1.7 GiB.                                      


## Update ID version

As suggested by Raph: The joint calling is usually done with pre-100 genomes dbSNP version 138, so lots missing dbSNP IDs for rare variants. You may want to update IDs using newer version dbSNP vcf file and then for variants still without dbSNP IDs create names for those, typically naming format is chr_pos_alleles. The bcftools annotate command can perform both these; of course want to update IDs first then rename where still missing.

#### prior to converting from vcf to plink helps to go ahead and include dx and dx ???

can format using plink2 sample file psam
#FID IID sex dx

hold over from plink1 sex; male=1, female=2, unknown/other=0

## Preprocessing for the purpose of QC 
According to Raph's suggestion, using plink instead of VSRQ.
Two versions of plink were used in Raph and Anni's notebook for different purpose. For example, multiallelics option is available in plink2, however, flags of --check-sex don't work plink2
1. Use plink2: Use the option of --make-pgen multiallelics=-, otherwise error occurs during plink trim from the bed file
2. Use plink1.9: Extract chrX for sexcheck

In [ ]:
# templates for dealing with multiallelic in plink2
# plink2 --vcf <input VCF> --make-pgen multiallelics=- --out tmp
# plink2 --pfile tmp --make-bed --out <final prefix>
# rm tmp.*

In [13]:
PLINK2='/Users/pengl7/Downloads/WGS/plink2'

In [27]:
#convert from vcf to plink2

input_vcf = f'{WRKDIR}/genotypes/{COHORT}.vcf.gz'
out_file_set = f'{WRKDIR}/genotypes/{COHORT}'

# !/Users/mooreank/Downloads/plink2 --vcf {input_vcf} --double-id \
# --update-sex /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt col-num=3 \
# --pheno /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt --pheno-col-nums 4 \
# --silent --allow-extra-chr --make-pgen --out {out_file_set}

!{PLINK2} --vcf {input_vcf} --double-id \
--silent --allow-extra-chr --make-pgen multiallelics=- --out {out_file_set}

# input_vcf = f'{WRKDIR}/genotypes/{COHORT}.chrX.gtonly.vcf.gz'
# out_file_set = f'{WRKDIR}/genotypes/{COHORT}.chrX'

# !/Users/mooreank/Downloads/plink2 --vcf {input_vcf} --double-id \
# --update-sex /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt col-num=3 \
# --pheno /Users/mooreank/Desktop/WGS/genotypes/ninds_eopd.txt --pheno-col-nums 4 \
# --silent --allow-extra-chr --make-pgen --out {out_file_set}

# !/Users/mooreank/Downloads/plink2 --vcf {input_vcf} --double-id \
# --silent --allow-extra-chr --make-pgen --out {out_file_set}

In [28]:
#check creation and logging
!ls -lth {WRKDIR}/genotypes/{COHORT}.*
!tail {WRKDIR}/genotypes/{COHORT}.*log

-rw-r--r--  1 pengl7  NIH\Domain Users   1.3K Aug 26 11:33 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.log
-rw-r--r--  1 pengl7  NIH\Domain Users    40M Aug 26 11:33 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.pgen
-rw-r--r--  1 pengl7  NIH\Domain Users   2.8G Aug 26 11:33 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.pvar
-rw-r--r--  1 pengl7  NIH\Domain Users   209B Aug 26 11:33 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.psam
-rw-r--r--  1 pengl7  NIH\Domain Users   1.7G Aug 26 10:06 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.vcf.gz
-rw-r--r--  1 pengl7  NIH\Domain Users   2.5M Aug 26 10:03 /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.vcf.gz.tbi
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax-temporary.psam.
9893835 variants loaded from
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax-temporary.pvar.
Note: No phenotype data present.
Writing /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.psam ... done.
Writing /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.pvar ... done.
Writing /

## Trim variants to QC set

what do geno, maf, and hwe mean?

- --geno Missing genotype rates: filters out all variants with missing call rates exceeding the provided value (default 0.1) to be removed, while --mind does the same for samples.
- --maf (minor allele frequency (MAF)) filters out all variants with allele frequency below the provided threshold (default 0.01)
- --hwe filters out all variants which have Hardy-Weinberg equilibrium exact test p-value below the provided threshold.

In [ ]:
INPUT_DIR='/Users/pengl7/Downloads/WGS/{COHORT}/genotypes/'

In [30]:
#trim variant to QC set
input_file_set = f'{WRKDIR}/genotypes/{COHORT}'
out_file_set = f'{WRKDIR}/qc/{COHORT}.geno05maf05hwe000001'

#!mkdir -p {WRKDIR}/qc

#!{PLINK2} --vcf {input_file_set} --make-pgen multiallelics=- --double-id --allow-extra-chr --out {out_file_set}
!{PLINK2} --pfile {input_file_set} --make-bed --geno 0.05 --maf 0.05 --hwe 0.000001 --out {out_file_set}

PLINK v2.00a2.3 64-bit (24 Jan 2020)           www.cog-genomics.org/plink/2.0/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001.log.
Options in effect:
  --geno 0.05
  --hwe 0.000001
  --maf 0.05
  --make-bed
  --out /Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001
  --pfile /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax

Start time: Wed Aug 26 11:51:35 2020
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 12 threads (change this with --threads).
8 samples (0 females, 0 males, 8 ambiguous; 8 founders) loaded from
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.psam.
10756762 variants loaded from
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.pvar.
Note: No phenotype data present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808

### Extract chrX for checksex at the step of --make-bed using opton of --chr X

In [46]:
#trim variant to QC set
input_file_set = f'{WRKDIR}/genotypes/{COHORT}'
out_file_set = f'{WRKDIR}/qc/{COHORT}.chrX.geno05maf05'

#!mkdir -p {WRKDIR}/qc

#!{PLINK2} --vcf {input_file_set} --make-pgen multiallelics=- --double-id --allow-extra-chr --out {out_file_set}
!{PLINK2} --pfile {input_file_set} --make-bed --chr X --geno 0.05 --maf 0.05 --out {out_file_set}

PLINK v2.00a2.3 64-bit (24 Jan 2020)           www.cog-genomics.org/plink/2.0/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.chrX.geno05maf05.log.
Options in effect:
  --chr X
  --geno 0.05
  --maf 0.05
  --make-bed
  --out /Users/pengl7/Downloads/WGS/Jax/qc/Jax.chrX.geno05maf05
  --pfile /Users/pengl7/Downloads/WGS/Jax/genotypes/Jax

Start time: Wed Aug 26 13:37:32 2020
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 12 threads (change this with --threads).
8 samples (0 females, 0 males, 8 ambiguous; 8 founders) loaded from
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.psam.
292398 out of 10756762 variants loaded from
/Users/pengl7/Downloads/WGS/Jax/genotypes/Jax.pvar.
Note: No phenotype data present.
Calculating allele frequencies... 18406285done.
--geno: 23084 variants removed due to missing genotype data.
23306 variants removed due to allele frequency threshold(s)
(--maf/--

In [107]:
%ls -lth {WRKDIR}/qc

total 1644656
-rw-r--r--  1 pengl7  NIH\Domain Users   3.0K Aug 26 15:20 Jax.king.related.log
-rw-r--r--  1 pengl7  NIH\Domain Users    46B Aug 26 15:20 Jax.king.relatedX.kin0
-rw-r--r--  1 pengl7  NIH\Domain Users    45B Aug 26 15:20 Jax.king.relatedX.kin
-rw-r--r--  1 pengl7  NIH\Domain Users   2.0K Aug 26 15:20 Jax.king.related.kin0
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0K Aug 26 14:05 Jax.het.log
-rw-r--r--  1 pengl7  NIH\Domain Users   774B Aug 26 14:05 Jax.het.het
-rw-r--r--  1 pengl7  NIH\Domain Users   172B Aug 26 14:05 Jax.het.nosex
-rw-r--r--  1 pengl7  NIH\Domain Users   1.1K Aug 26 13:59 Jax.missing.log
-rw-r--r--  1 pengl7  NIH\Domain Users   152M Aug 26 13:59 Jax.missing.vmiss
-rw-r--r--  1 pengl7  NIH\Domain Users   302B Aug 26 13:59 Jax.missing.smiss
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0K Aug 26 13:53 Jax.sex2.log
-rw-r--r--  1 pengl7  NIH\Domain Users   774B Aug 26 13:53 Jax.sex2.sexcheck
-rw-r--r--  1 pengl7  NIH\Domain Users   172B Aug 26 13:53 Jax.sex2.n

### Check sex

## Plink2 doesn't have the tag --check-sex, so change to plink1.9
PLINK v2.00a2.3 64-bit (24 Jan 2020)           www.cog-genomics.org/plink/2.0/
Error: Unrecognized flag ('--check-sex').

In [54]:
%ls -lth /Users/pengl7/Downloads/WGS/plink_mac_20200616/

total 4464
-rwxr-xr-x@ 1 pengl7  NIH\Domain Users   2.1M Jul 12 13:05 plink*
-rw-r--r--@ 1 pengl7  NIH\Domain Users    58B Mar 22  2019 toy.ped
-rwxr-xr-x@ 1 pengl7  NIH\Domain Users    18K Mar  5  2019 prettify*
-rw-r--r--@ 1 pengl7  NIH\Domain Users    34K Feb 19  2018 LICENSE
-rw-r--r--@ 1 pengl7  NIH\Domain Users    27B Feb 19  2018 toy.map


In [51]:
# Use plink1.9
plink="/Users/pengl7/Downloads/WGS/plink_mac_20200616/plink"

### Using one of the two methods below (either my method or Raph's)

In [57]:
!{plink} --bfile {WRKDIR}/qc/{COHORT}.chrX.geno05maf05 --check-sex 0.25 0.75 --out {WRKDIR}/qc/{COHORT}.sex2

PLINK v1.90p 64-bit (16 Jun 2020)              www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex2.log.
Options in effect:
  --bfile /Users/pengl7/Downloads/WGS/Jax/qc/Jax.chrX.geno05maf05
  --check-sex 0.25 0.75
  --out /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex2

16384 MB RAM detected; reserving 8192 MB for main workspace.
246008 variants loaded from .bim file.
8 people (0 males, 0 females, 8 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex2.nosex
.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 8 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
246008 variants and 8 people pass f

In [110]:
%%bash -s "$COHORT" "$WRKDIR"
#check gender
COHORT=${1}
WRKDIR=${2}"/qc"
plink=${3}
#hg38 non-PAR
awk '$4 > 2800000 && $4 < 155700000 {print $2}' ${WRKDIR}/${COHORT}.chrX.geno05maf05.bim \
    > ${WRKDIR}/${COHORT}.chrX.list

plink1.9 --bfile ${WRKDIR}/${COHORT}.chrX.geno05maf05 --extract ${WRKDIR}/${COHORT}.chrX.list \
--check-sex 0.25 0.75 --out ${WRKDIR}/${COHORT}.sex

PLINK v1.90p 64-bit (16 Jun 2020)              www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex.log.
Options in effect:
  --bfile /Users/pengl7/Downloads/WGS/Jax/qc/Jax.chrX.geno05maf05
  --check-sex 0.25 0.75
  --extract /Users/pengl7/Downloads/WGS/Jax/qc/Jax.chrX.list
  --out /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex

16384 MB RAM detected; reserving 8192 MB for main workspace.
246008 variants loaded from .bim file.
8 people (0 males, 0 females, 8 ambiguous) loaded from .fam.
Ambiguous sex IDs written to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.sex.nosex .
--extract: 245947 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 8 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%2

In [58]:
#check sex test results
sexcheck_df = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.sex.sexcheck', sep='\s+')
print(f'sexcheck shape is {sexcheck_df.shape}')

fail_sexcheck_df = sexcheck_df.loc[sexcheck_df['STATUS'] == 'PROBLEM']
print(f'number of samples failing sexcheck {fail_sexcheck_df.shape[0]}')
print(fail_sexcheck_df)

fail_sexcheck_df = sexcheck_df.loc[(sexcheck_df['STATUS'] == 'PROBLEM') & \
                                   (sexcheck_df['PEDSEX'] != 0)]
print(f'number of samples failing sexcheck excluding missing info {fail_sexcheck_df.shape}')
print(fail_sexcheck_df)

sexcheck shape is (8, 6)
number of samples failing sexcheck 8
              FID             IID  PEDSEX  SNPSEX   STATUS       F
0  KOLF2-ARID2-A2  KOLF2-ARID2-A2       0       1  PROBLEM  0.8124
1        KUCG3-C1        KUCG3-C1       0       1  PROBLEM  0.7902
2       LNGPI1-C1       LNGPI1-C1       0       1  PROBLEM  0.8041
3        NCRM1-C6        NCRM1-C6       0       1  PROBLEM  0.8177
4        NCRM5-C5        NCRM5-C5       0       1  PROBLEM  0.8045
5    NN0003932-C3    NN0003932-C3       0       1  PROBLEM  0.8132
6    NN0004297-C1    NN0004297-C1       0       1  PROBLEM  0.7929
7         PGP1-C2         PGP1-C2       0       1  PROBLEM  0.8233
number of samples failing sexcheck excluding missing info (0, 6)
Empty DataFrame
Columns: [FID, IID, PEDSEX, SNPSEX, STATUS, F]
Index: []


In [62]:
#check missingness
!{PLINK2} --bfile {WRKDIR}/qc/{COHORT}.geno05maf05hwe000001 --missing --autosome \
--out {WRKDIR}/qc/{COHORT}.missing

PLINK v2.00a2.3 64-bit (24 Jan 2020)           www.cog-genomics.org/plink/2.0/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /Users/pengl7/Downloads/WGS/Jax/qc/Jax.missing.log.
Options in effect:
  --autosome
  --bfile /Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001
  --missing
  --out /Users/pengl7/Downloads/WGS/Jax/qc/Jax.missing

Start time: Wed Aug 26 13:59:03 2020
16384 MiB RAM detected; reserving 8192 MiB for main workspace.
Using up to 12 threads (change this with --threads).
8 samples (0 females, 0 males, 8 ambiguous; 8 founders) loaded from
/Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001.fam.
9323608 out of 9601254 variants loaded from
/Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001.bim.
Note: No phenotype data present.
Calculating sample missingness rates... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798

In [61]:
%ls -lth {WRKDIR}/qc/{COHORT}.missing*

-rw-r--r--  1 pengl7  NIH\Domain Users   1.1K Aug 26 13:56 /Users/pengl7/Downloads/WGS/Jax/qc/Jax.missing.log
-rw-r--r--  1 pengl7  NIH\Domain Users   152M Aug 26 13:56 /Users/pengl7/Downloads/WGS/Jax/qc/Jax.missing.vmiss
-rw-r--r--  1 pengl7  NIH\Domain Users   302B Aug 26 13:56 /Users/pengl7/Downloads/WGS/Jax/qc/Jax.missing.smiss


In [63]:
#check missingness results
misstest_df = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.missing.smiss', sep='\s+')

#find failed
misstest_failed_df = misstest_df.loc[misstest_df['F_MISS'] > 0.05]

print(f'number samples failing missingness test {misstest_failed_df.shape[0]}')

print(misstest_failed_df)

number samples failing missingness test 0
Empty DataFrame
Columns: [#FID, IID, MISSING_CT, OBS_CT, F_MISS]
Index: []


### check het rates

In [66]:
#check het rates, plink2 doesn't work
!{plink} --bfile {WRKDIR}/qc/{COHORT}.geno05maf05hwe000001 --het --autosome \
--silent --out {WRKDIR}/qc/{COHORT}.het

In [67]:
#check het rate results

###2 failes

hets_df = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.het.het', sep='\s+')

#find failed
hets_failed_df = hets_df.loc[(hets_df['F'] > 0.15) | (hets_df['F'] < -0.15)]

print(f'number samples failing heterzygosity check {hets_failed_df.shape[0]}')

print(hets_failed_df)

number samples failing heterzygosity check 0
Empty DataFrame
Columns: [FID, IID, O(HOM), E(HOM), N(NM), F]
Index: []


### check for relatedness, including duplicates

In [76]:
%ls -lth /Users/pengl7/Downloads/WGS/plink_mac_20200616/

total 4464
-rwxr-xr-x@ 1 pengl7  NIH\Domain Users   2.1M Jul 12 13:05 plink*
-rw-r--r--@ 1 pengl7  NIH\Domain Users    58B Mar 22  2019 toy.ped
-rwxr-xr-x@ 1 pengl7  NIH\Domain Users    18K Mar  5  2019 prettify*
-rw-r--r--@ 1 pengl7  NIH\Domain Users    34K Feb 19  2018 LICENSE
-rw-r--r--@ 1 pengl7  NIH\Domain Users    27B Feb 19  2018 toy.map


In [77]:
#!cp /Users/pengl7/Downloads/WGS/king /usr/local/bin/king
!cp /Users/pengl7/Downloads/WGS/plink2 /usr/local/bin/plink2

!cp /Users/pengl7/Downloads/WGS/plink_mac_20200616/plink /usr/local/bin/plink1.9

In [88]:
!cp /Users/pengl7/Downloads/WGS/king/king /usr/local/bin/king

In [89]:
!which king


/usr/local/bin/king


In [80]:
!which plink1.9

/usr/local/bin/plink1.9


## KING is a toolset to explore genotype data from a genome-wide association study (GWAS) or a sequencing project. 

The latest version is KING 2.2.5 available on June 5, 2020. KING can be used to check family relationship and flag pedigree errors by estimating kinship coefficients and inferring IBD segments for all pairwise relationships. Unrelated pairs can be precisely separated from close relatives with no false positives, with accuracy up to 3rd- or 4th-degree (depending on array or WGS) for --related and --ibdseg analyses, and up to 2nd-degree for --kinship analysis.

http://people.virginia.edu/~wc9c/KING/manual.html

In [96]:
# king_related_cmd = f'nohup king --related -b {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001.bed \
# --cpus $(nproc) --degree 2 --prefix {WRKDIR}/qc/{COHORTBUILD}.king.related \
# > {WRKDIR}/qc/{COHORTBUILD}.king.related.log &'

# king_related_cmd = f'nohup /Users/mooreank/Downloads/king --related -b {WRKDIR}/qc/{COHORTBUILD}.geno05maf05hwe000001.bed \
#  --degree 2 --prefix {WRKDIR}/qc/{COHORTBUILD}.king.related \
# > {WRKDIR}/qc/{COHORTBUILD}.king.related.log &'


king_related_cmd = f'king --related -b {WRKDIR}/qc/{COHORT}.geno05maf05hwe000001.bed \
 --degree 2 --prefix {WRKDIR}/qc/{COHORT}.king.related \
> {WRKDIR}/qc/{COHORT}.king.related.log &'

print('#run these commands at terminal:\n')
print(king_related_cmd)

#run these commands at terminal:

king --related -b /Users/pengl7/Downloads/WGS/Jax/qc/Jax.geno05maf05hwe000001.bed  --degree 2 --prefix /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.related > /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.related.log &


In [97]:
#check the king log
!tail -n 16 {WRKDIR}/qc/{COHORT}.king.related.log

	--kinship
	--prefix /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.related

Relationship inference across families starts at Wed Aug 26 15:20:31 2020
6 CPU cores are used.
                                         ends at Wed Aug 26 15:20:31 2020
Between-family kinship data saved in file /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.related.kin0
Note --kinship --degree <n> can filter & speed up the kinship computing.

X-chromosome analysis...
X-chromosome genotypes stored in 3844 64-bit words for each of 8 individuals.
Within-family kinship data saved in file /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.relatedX.kin
Relationship inference across families starts at Wed Aug 26 15:20:31 2020
                                         ends at Wed Aug 26 15:20:31 2020
Between-family kinship data saved in file /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.relatedX.kin0
KING ends at Wed Aug 26 15:20:31 2020


### For the duplicates see which ones also failed sexcheck, ie clues how to correctly resolve duplicates¶

In [99]:
%ls -lth {WRKDIR}/qc

total 1644656
-rw-r--r--  1 pengl7  NIH\Domain Users   3.0K Aug 26 15:20 Jax.king.related.log
-rw-r--r--  1 pengl7  NIH\Domain Users    46B Aug 26 15:20 Jax.king.relatedX.kin0
-rw-r--r--  1 pengl7  NIH\Domain Users    45B Aug 26 15:20 Jax.king.relatedX.kin
-rw-r--r--  1 pengl7  NIH\Domain Users   2.0K Aug 26 15:20 Jax.king.related.kin0
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0K Aug 26 14:05 Jax.het.log
-rw-r--r--  1 pengl7  NIH\Domain Users   774B Aug 26 14:05 Jax.het.het
-rw-r--r--  1 pengl7  NIH\Domain Users   172B Aug 26 14:05 Jax.het.nosex
-rw-r--r--  1 pengl7  NIH\Domain Users   1.1K Aug 26 13:59 Jax.missing.log
-rw-r--r--  1 pengl7  NIH\Domain Users   152M Aug 26 13:59 Jax.missing.vmiss
-rw-r--r--  1 pengl7  NIH\Domain Users   302B Aug 26 13:59 Jax.missing.smiss
-rw-r--r--  1 pengl7  NIH\Domain Users   1.0K Aug 26 13:53 Jax.sex2.log
-rw-r--r--  1 pengl7  NIH\Domain Users   774B Aug 26 13:53 Jax.sex2.sexcheck
-rw-r--r--  1 pengl7  NIH\Domain Users   172B Aug 26 13:53 Jax.sex2.n

In [101]:
#get the sex problems
sexcheck = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.sex.sexcheck',sep='\s+')
sexmismatch = sexcheck.loc[(sexcheck['PEDSEX'] != 0) & (sexcheck['STATUS'] == 'PROBLEM')]
sexmismatch

,FID,IID,PEDSEX,SNPSEX,STATUS,F


In [106]:
!cat {WRKDIR}/qc/{COHORT}.king.related.kin0 | column -t

FID1            ID1             FID2          ID2           N_SNP    HetHet  IBS0    Kinship
KOLF2-ARID2-A2  KOLF2-ARID2-A2  KUCG3-C1      KUCG3-C1      9323608  0.1262  0.0526  0.0291
KOLF2-ARID2-A2  KOLF2-ARID2-A2  LNGPI1-C1     LNGPI1-C1     9323608  0.1249  0.0521  0.0302
KOLF2-ARID2-A2  KOLF2-ARID2-A2  NCRM1-C6      NCRM1-C6      9323608  0.1229  0.0527  0.0244
KOLF2-ARID2-A2  KOLF2-ARID2-A2  NCRM5-C5      NCRM5-C5      9323608  0.1227  0.0529  0.0235
KOLF2-ARID2-A2  KOLF2-ARID2-A2  NN0003932-C3  NN0003932-C3  9323608  0.1225  0.0534  0.0202
KOLF2-ARID2-A2  KOLF2-ARID2-A2  NN0004297-C1  NN0004297-C1  9323608  0.1235  0.0528  0.0252
KOLF2-ARID2-A2  KOLF2-ARID2-A2  PGP1-C2       PGP1-C2       9323608  0.1218  0.0534  0.0221
KUCG3-C1        KUCG3-C1        LNGPI1-C1     LNGPI1-C1     9323608  0.1251  0.0517  0.0292
KUCG3-C1        KUCG3-C1        NCRM1-C6      NCRM1-C6      9323608  0.1230  0.0526  0.0225
KUCG3-C1        KUCG3-C1        NCRM5-C5      NCRM5-C5      9323608  0.1239  0.

In [105]:
!wc -l {WRKDIR}/qc/{COHORT}.king.related.kin0

      29 /Users/pengl7/Downloads/WGS/Jax/qc/Jax.king.related.kin0


In [102]:
#get the duplicates
rels_df = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.king.related.kin0',sep='\s+')
print(f'number of related pairs {rels_df.shape[0]}')
print(rels_df['InfType'].value_counts())

number of related pairs 28


KeyError: 'InfType'

In [98]:
#get the sex problems
sexcheck = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.sex.sexcheck',sep='\s+')
sexmismatch = sexcheck.loc[(sexcheck['PEDSEX'] != 0) & (sexcheck['STATUS'] == 'PROBLEM')]

#get the duplicates
rels_df = pd.read_csv(f'{WRKDIR}/qc/{COHORT}.king.related.kin0',sep='\s+')
print(f'number of related pairs {rels_df.shape[0]}')
print(rels_df['InfType'].value_counts())

#subset just the dups
dups_df = rels_df.loc[rels_df['InfType'] == 'DUP/MZ']

print(f'\njust the DUP/MZ {dups_df.shape[0]}')
print(dups_df['InfType'].value_counts())

print('\nnumber of samples in that failed sexcheck and also a duplicate')
print(len(set(sexmismatch['IID']) & (set(rels_df['ID1']) | set(rels_df['ID2']))))


number of related pairs 28


KeyError: 'InfType'

In [5]:
%%bash -s "$COHORT_BUCKET"

COHORT_BUCKET=${1}

gsutil ls ${COHORT_BUCKET}/ubams | wc -l
echo ${COHORT_BUCKET}/ubams
gsutil ls ${COHORT_BUCKET}/ubams


       8
gs://test-7cee72c0e768/Jax/ubams
gs://test-7cee72c0e768/Jax/ubams/KOLF2-ARID2-A2/
gs://test-7cee72c0e768/Jax/ubams/KUCG3-C1/
gs://test-7cee72c0e768/Jax/ubams/LNGPI1-C1/
gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/
gs://test-7cee72c0e768/Jax/ubams/NCRM5-C5/
gs://test-7cee72c0e768/Jax/ubams/NN0003932-C3/
gs://test-7cee72c0e768/Jax/ubams/NN0004297-C1/
gs://test-7cee72c0e768/Jax/ubams/PGP1-C2/


In [6]:
!gsutil ls gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/

gs://test-7cee72c0e768/Jax/ubams/NCRM1-C6/NCRM1-C6_GT20-02407_L003_200221.unmapped.bam
